In [ ]:
%pylab inline
from astropy.io import fits as pyfits
from astropy.table import Table,join
import numpy as np
from astropy.io.fits import Column
from datetime import datetime
import matplotlib.pyplot as plt
import requests

warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=UserWarning, append=True);

In [15]:
# Load data from Dropbox folder instead of clogging up Github

def download_from_dropbox(url):
    
    local_filename = "../{:}".format(url.split("/")[-1].split("?")[0])
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                f.flush()
            
    return local_filename

In [ ]:
# 3 tables:
# Table 1 - gz_hst_table_raw_and_weighted_votes.fits; contains all the vote data except for goods_full, but I hope to add that as soon as I get that data.
votes_fname = download_from_dropbox("https://www.dropbox.com/s/97e0p6nla1zxqxc/gz_hst_table_raw_and_weighted_votes.fits?dl=1")
votes_data = Table.read(votes_fname)

# Table 2 - lo_hi_limits.fits: just need low, high limits on p_features (2 columns). 
lohi_fname = download_from_dropbox("https://www.dropbox.com/s/kmwoj3llzjvqey4/lo_hi_limits.fits?dl=1")
lohi_data = Table.read(lohi_fname)

#Table 3 - gzh_t01_z_mu_categorized.fits, want z, mu, and category information (3 columns)
meta_fname = download_from_dropbox("https://www.dropbox.com/s/l80ibbm8y7rf2tw/gzh_t01_z_mu_categorized.fits?dl=1")
meta_data = Table.read(meta_fname)


In [17]:
lohi_data.remove_column('Z_BEST')
lohi_data.remove_column('MU_HI')
lohi_data.remove_column('t01_smooth_or_features_a02_features_or_disk_weighted_fraction')

meta_data.remove_column('t01_smooth_or_features_a02_features_or_disk_weighted_fraction')

In [18]:
#Join Table 2 and Table 3- will have z, mu, low/hi limits on p_features
lohi_meta = join(meta_data, lohi_data, keys='OBJNO',join_type='left')

In [19]:
len(lohi_meta)

181101

In [20]:
#strings have empty spaces in objno and correctable_category columns - get rid of them! 
for i in range(0,len(votes_data['OBJNO'])):
    votes_data['OBJNO'][i]=votes_data['OBJNO'][i].strip()
    lohi_meta['OBJNO'][i]=lohi_meta['OBJNO'][i].strip()
    lohi_meta['Correctable_Category'][i]=lohi_meta['Correctable_Category'][i].strip()

In [21]:
#join Table 1 (votes + Imaging) with the other stuff:

input_table=join(votes_data,lohi_meta,keys='OBJNO')

In [22]:
#Set path for output
fname = '/home/mel/Dropbox/gzhubble/hubble_files/catalog_debiasing_files/input_for_hubble_debiased_catalog.fits'

input_table.write(fname,overwrite=True)
